In [128]:
from dotenv import load_dotenv
import requests
import json
import os
from sqlalchemy import create_engine, inspect
from sqlalchemy.schema import CreateSchema, DropTable, _DropView
from sqlalchemy.sql import text
import pandas as pd

## Consuming API

In [102]:
"""
    ...
"""
load_dotenv()
api_token = os.getenv("API_Token")

"""
    /v4/competitions/{id}/teams
    List all teams for a particular competition.
"""
headers = {'X-Auth-Token': api_token}


# GET ALL COMPETITIONS --+ 10
uri = "https://api.football-data.org/v4/competitions"
arr_competition_id = []
response = requests.get(uri,headers=headers)
if response.status_code == 200:   
    with open('./data_json/competitions.json', 'w') as f:
        json.dump(response.json(), f, ensure_ascii=False, indent=4)
    for competition in response.json()['competitions']:
        arr_competition_id.append(competition['id'])
else:
    print('Error')

In [70]:
print(arr_competition_id)

[2013, 2016, 2021, 2001, 2018, 2015, 2002, 2019, 2003, 2017, 2152, 2014, 2000]


In [126]:
json_competition_teams = []
for competition_id in arr_competition_id:
    uri_competition_teams = f"http://api.football-data.org/v4/competitions/{competition_id}/teams"
    response = requests.get(uri_competition_teams,headers=headers)
    if response.status_code == 200:
        json_competition_teams.append(response.json())
    else:
        print('Error' + str(response.status_code))

with open('./data_json/competition_teams.json', 'w') as f:
    json.dump(json_competition_teams, f, ensure_ascii=False, indent=4)


Error429
Error429


In [ ]:
"""
# GET ALL TEAMS --+ 50
uri = "https://api.football-data.org/v4/teams"
response = requests.get(uri,headers=headers)
if response.status_code == 200:
    print(len(response.json()['teams'])) 
    #for match in response.json()['teams']:
    #    print(match)
else:
    print('Error')
"""

In [ ]:
# https://docs.football-data.org/general/v4/errors.html#_429_too_many_requests

## Inserting data into SQLITE

In [135]:
engine = create_engine('sqlite:///./data/db_bcg_project.db') # ensure this is the correct path for the sqlite file. 

In [202]:
data_competitions = None
with open('./data_json/competitions.json', 'r') as file:
    data_competitions = json.loads(file.read())

pd_series_competitions = pd.Series(data_competitions['competitions'])
# pd_series_competitions.head()

df_competitions = pd_series_competitions.to_frame()
df_competitions = pd.json_normalize(df_competitions[0])
df_competitions = df_competitions[['id','name']]
# df.head()
df_competitions.head(n=0).to_sql(name='dim_competitions', con=engine, index=False, if_exists='replace')
df_competitions.to_sql(name='dim_competitions', con=engine, index=False, if_exists='append')


13

In [199]:
data_competition_teams = None
with open('./data_json/competition_teams.json', 'r') as file:
    data_competition_teams = json.loads(file.read())

pd_serie_competition_teams = pd.Series(data_competition_teams)
pd_serie_competition_teams.head()

df_competition_teams = pd_serie_competition_teams.to_frame()
df_competition_teams = pd.json_normalize(df_competition_teams[0],'teams',[['competition','id'],['competition','name']])

df_competition_teams = df_competition_teams.rename(columns={'competition.id':'fact_competitions','competition.name':'competition_name','id':'team_id'})

df_competition_teams = df_competition_teams[['team_id','name','shortName','tla','fact_competitions','competition_name']]
df_competition_teams_fact = df_competition_teams[['fact_competitions','team_id']]

df_teams_dim = df_competition_teams[['team_id','name']]
df_teams_dim = df_teams_dim.rename(columns={'team_id':'id'})
df_teams_dim = df_teams_dim.drop_duplicates()


In [201]:
# fact
df_competition_teams_fact.head(n=0).to_sql(name='fact_competitions', con=engine, index=False, if_exists='replace')
df_competition_teams_fact.to_sql(name='fact_competitions', con=engine, index=False, if_exists='append')


259

In [200]:
# dimension
df_teams_dim.head(n=0).to_sql(name='dim_teams', con=engine, index=False, if_exists='replace')
df_teams_dim.to_sql(name='dim_teams', con=engine, index=False, if_exists='append')

229

## Commands

```bash
conda env create -f environment.yaml
conda activate bcg_project
conda deactivate

sudo apt install sqlite3
sqlite3 db_name.db
```